# Singapore Traffic Simulation — LTA × OSMnx × NEA Rainfall (Interactive)

Interactive, data-driven simulation using:
- **OSMnx** road network
- **LTA DataMall** Traffic Speed Bands v4 + Traffic Incidents
- **NEA** (data.gov.sg) Rainfall (5-min)

Controls: **demand (time-of-day)** and **rainfall** sliders → simulated speeds & congestion maps.


## 0) Setup
```bash
conda create -n sgtraffic python=3.11 -y
conda activate sgtraffic
pip install numpy pandas geopandas shapely pyproj rtree osmnx networkx requests plotly folium tqdm ipywidgets statsmodels
jupyter nbextension enable --py widgetsnbextension
```


In [24]:
import os
LTA_ACCOUNT_KEY = os.getenv('LTA_ACCOUNT_KEY', '99v+LNo8S4qid/MgSTDrkA==')
NEA_API_KEY = os.getenv('DATA_GOV_SG_API_KEY', '')  # optional
#assert LTA_ACCOUNT_KEY and LTA_ACCOUNT_KEY!='99v+LNo8S4qid/MgSTDrkA==', 'Please set LTA_ACCOUNT_KEY.'

In [8]:
import osmnx as ox, networkx as nx
ox.settings.use_cache = True
G = ox.graph_from_place('Singapore', network_type='drive')
#G = ox.simplify_graph(G)
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
#ox.save_graphml(G, 'sg_drive.graphml')
print('Nodes:', G.number_of_nodes(), 'Edges:', G.number_of_edges())

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Nodes: 24135 Edges: 45951


In [9]:
edges_gdf = ox.graph_to_gdfs(G, nodes=False, fill_edge_geometry=True)
edges_gdf.head()

osmid  \
u        v          key                                                      
25451929 6749812859 0    [49961799, 718881456, 741164883, 754786742, 17...   
25455287 1637003462 0                                            150829205   
         2521018789 0                     [633215386, 74607482, 635109319]   
26777521 172424179  0                    [481674707, 654734557, 633179143]   
         1889379421 0         [643413273, 559564482, 640203517, 640203518]   

                               highway   lanes maxspeed  \
u        v          key                                   
25451929 6749812859 0         motorway       5       70   
25455287 1637003462 0    motorway_link       1       50   
         2521018789 0         motorway  [3, 2]       90   
26777521 172424179  0          primary       2       40   
         1889379421 0      residential       2       50   

                                                  name  oneway  ref reversed  \
u        v          key                                                        
25451929 6749812859 0               East Coast Parkway    True  ECP    False   
25455287 1637003462 0    Kallang Paya Lebar Expressway    True  NaN    False   
         2521018789 0               East Coast Parkway    True  ECP    False   
26777521 172424179  0                      Grange Road    True  NaN    False   
         1889379421 0                    Cuscaden Road   False  NaN    False   

                             length bridge  \
u        v          key                      
25451929 6749812859 0    765.027747    yes   
25455287 1637003462 0    629.055082    NaN   
         2521018789 0    652.575579    NaN   
26777521 172424179  0    113.995770    NaN   
         1889379421 0     86.667744    NaN   

                                                                  geometry  \
u        v          key                                                      
25451929 6749812859 0    LINESTRING (103.87254 1.29523, 103.87103 1.295...   
25455287 1637003462 0    LINESTRING (103.874 1.29544, 103.87413 1.2955,...   
         2521018789 0    LINESTRING (103.874 1.29544, 103.87438 1.29544...   
26777521 172424179  0    LINESTRING (103.82357 1.30398, 103.82359 1.303...   
         1889379421 0    LINESTRING (103.82357 1.30398, 103.82372 1.304...   

                         speed_kph  travel_time tunnel junction access width  
u        v          key                                                       
25451929 6749812859 0         70.0    39.344284    NaN      NaN    NaN   NaN  
25455287 1637003462 0         50.0    45.291966    NaN      NaN    NaN   NaN  
         2521018789 0         90.0    26.103023    NaN      NaN    NaN   NaN  
26777521 172424179  0         40.0    10.259619    NaN      NaN    NaN   NaN  
         1889379421 0         50.0     6.240078    NaN      NaN    NaN   NaN

In [17]:
edges_gdf

,u,v,key,osmid,highway,lanes,maxspeed,name,oneway,ref,reversed,length,bridge,geometry,speed_kph,travel_time,tunnel,junction,access,width
0,25451929,6749812859,0,"[49961799, 718881456, 741164883, 754786742, 17...",motorway,5,70,East Coast Parkway,True,ECP,False,765.027747,yes,"LINESTRING (103.87254 1.29523, 103.87103 1.295...",70.000000,39.344284,NaN,NaN,NaN,NaN
1,25455287,1637003462,0,150829205,motorway_link,1,50,Kallang Paya Lebar Expressway,True,NaN,False,629.055082,NaN,"LINESTRING (103.874 1.29544, 103.87413 1.2955,...",50.000000,45.291966,NaN,NaN,NaN,NaN
2,25455287,2521018789,0,"[633215386, 74607482, 635109319]",motorway,"[3, 2]",90,East Coast Parkway,True,ECP,False,652.575579,NaN,"LINESTRING (103.874 1.29544, 103.87438 1.29544...",90.000000,26.103023,NaN,NaN,NaN,NaN
3,26777521,172424179,0,"[481674707, 654734557, 633179143]",primary,2,40,Grange Road,True,NaN,False,113.995770,NaN,"LINESTRING (103.82357 1.30398, 103.82359 1.303...",40.000000,10.259619,NaN,NaN,NaN,NaN
4,26777521,1889379421,0,"[643413273, 559564482, 640203517, 640203518]",residential,2,50,Cuscaden Road,False,NaN,False,86.667744,NaN,"LINESTRING (103.82357 1.30398, 103.82372 1.304...",50.000000,6.240078,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45946,12649794025,5599381219,0,1365971181,secondary_link,NaN,NaN,NaN,True,NaN,False,39.616302,NaN,"LINESTRING (103.94858 1.31706, 103.94852 1.317...",49.835526,2.861788,NaN,NaN,NaN,NaN
45947,12649794026,11981135278,0,"[1365461841, 1292410625, 1292410627, 1422097547]","[secondary, tertiary]",NaN,NaN,"[Bayshore Drive, Bayshore Avenue]",True,NaN,False,538.107748,NaN,"LINESTRING (103.94875 1.31705, 103.94882 1.316...",53.900229,35.940253,NaN,NaN,NaN,NaN
45948,12649794031,243486497,0,175959563,secondary,2,50,Upper East Coast Road,True,NaN,False,43.119502,NaN,"LINESTRING (103.94898 1.3175, 103.94883 1.3174...",50.000000,3.104604,NaN,NaN,NaN,NaN
45949,12649794031,12649794026,0,1365971182,secondary_link,NaN,NaN,NaN,True,NaN,False,57.805331,NaN,"LINESTRING (103.94898 1.3175, 103.94888 1.3173...",49.835526,4.175720,NaN,NaN,NaN,NaN


In [25]:
import os, time, datetime as dt
import requests, pandas as pd

# === Keys ===
# LTA_ACCOUNT_KEY = os.getenv("LTA_ACCOUNT_KEY", "").strip()
# if not LTA_ACCOUNT_KEY:
#     print("WARNING: LTA_ACCOUNT_KEY is not set; requests will fail.")
HEADERS_LTA = {"AccountKey": LTA_ACCOUNT_KEY, "accept": "application/json"}

NEA_API_KEY = os.getenv("DATA_GOV_SG_API_KEY", "").strip()
HEADERS_NEA = {"api-key": NEA_API_KEY} if NEA_API_KEY else {}

# === Endpoints ===
URL_TSB = "https://datamall2.mytransport.sg/ltaodataservice/v4/TrafficSpeedBands"
URL_INC = "https://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents"
URL_RAIN = "https://api.data.gov.sg/v1/environment/rainfall"

# === Robust pagination: follow @odata.nextLink if present, else fall back to $skip ===
def fetch_paginated_all(base_url, headers, page_size=500, max_pages=400, sleep_s=0.25):
    rows, next_url, used_skip, skip = [], base_url, False, 0
    for _ in range(max_pages):
        if used_skip:
            next_url = f"{base_url}?$skip={skip}"
        resp = requests.get(next_url, headers=headers, timeout=30)
        resp.raise_for_status()
        data = resp.json()
        vals = data.get("value", [])
        rows.extend(vals)

        # Prefer server-provided nextLink
        nxt = data.get("@odata.nextLink") or data.get("odata.nextLink")
        if nxt:
            next_url = nxt
            used_skip = False
        else:
            used_skip = True
            # Stop if we got less than a full page
            if len(vals) < page_size:
                break
            skip += page_size

        time.sleep(sleep_s)  # be nice to the API
    return pd.json_normalize(rows)

# === Fetch all TSB and Incidents ===
tsb = fetch_paginated_all(URL_TSB, HEADERS_LTA)
inc = fetch_paginated_all(URL_INC, HEADERS_LTA)

# De-duplicate conservatively (keep latest occurrence)
tsb = tsb.drop_duplicates()
inc = inc.drop_duplicates()

# Map to observed speed (km/h)
if {"MinimumSpeed","MaximumSpeed"}.issubset(tsb.columns):
    tsb["speed_kph_obs"] = (pd.to_numeric(tsb["MinimumSpeed"], errors="coerce")
                            + pd.to_numeric(tsb["MaximumSpeed"], errors="coerce")) / 2.0
else:
    band_mid = {1:5, 2:15, 3:25, 4:35, 5:45, 6:55, 7:65, 8:75}
    tsb["speed_kph_obs"] = pd.to_numeric(tsb.get("SpeedBand"), errors="coerce").map(band_mid)

# Helpful numeric conversions (for geometry building later)
for c in ["StartLat","StartLon","EndLat","EndLon"]:
    if c in tsb.columns:
        tsb[c] = pd.to_numeric(tsb[c], errors="coerce")

print(f"TSB rows (all pages): {len(tsb):,}")
print(f"Incidents rows (all pages): {len(inc):,}")

# === NEA rainfall (optional; snapshot at closest available time) ===
params = {"date_time": dt.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S")}
try:
    jr = requests.get(URL_RAIN, params=params, headers=HEADERS_NEA, timeout=30).json()
    readings = pd.json_normalize(jr["items"][0]["readings"])
    stations = pd.json_normalize(jr["metadata"]["stations"])
    rain = (readings.rename(columns={"value":"rain_mm"})[["station_id","rain_mm"]]
            .merge(stations[["id","name","device_id","location.latitude","location.longitude"]],
                   left_on="station_id", right_on="id", how="left"))
    # numeric rain
    rain["rain_mm"] = pd.to_numeric(rain["rain_mm"], errors="coerce")
    print(f"NEA rainfall stations: {len(rain):,}")
except Exception as e:
    print("NEA rainfall fetch failed (continuing without it):", e)
    rain = pd.DataFrame()

# Peek
display(tsb.head(3))
display(inc.head(3))
display(rain.head(3) if not rain.empty else pd.DataFrame({"rain_mm":[]}))


TSB rows (all pages): 143,787
Incidents rows (all pages): 17
NEA rainfall stations: 58


/var/folders/bk/pgnkjj9d7gq27dnjs3fnbkc80000gn/T/ipykernel_95878/2277556875.py:70: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



,LinkID,RoadName,RoadCategory,SpeedBand,MinimumSpeed,MaximumSpeed,StartLon,StartLat,EndLon,EndLat,speed_kph_obs
0,2,NARAYANAN CHETTY ROAD,5,4,30,39,103.838305,1.292060,103.838331,1.292044,34.5
1,3,NARAYANAN CHETTY ROAD,5,1,0,9,103.838523,1.292320,103.838419,1.292174,4.5
2,4,NARAYANAN CHETTY ROAD,5,1,0,9,103.838419,1.292174,103.838358,1.292083,4.5


,Type,Latitude,Longitude,Message
0,Roadwork,1.295799,103.888701,(24/10)11:27 Road Works on ECP (towards Changi...
1,Roadwork,1.333233,103.814865,(24/10)11:22 Road Works on PIE (towards Tuas) ...
2,Roadwork,1.371633,103.724179,(24/10)11:08 Road Works on KJE (towards PIE) a...


,station_id,rain_mm,id,name,device_id,location.latitude,location.longitude
0,S224,0,S224,Airport Boulevard,S224,1.34392,103.98409
1,S77,0,S77,Alexandra Road,S77,1.29370,103.81250
2,S216,0,S216,Ang Mo Kio Avenue 10,S216,1.36019,103.85335


In [26]:
import geopandas as gpd
from shapely.geometry import LineString
edges_gdf = ox.graph_to_gdfs(G, nodes=False, fill_edge_geometry=True)
if not {'u','v','key'}.issubset(edges_gdf.columns):
    edges_gdf = edges_gdf.reset_index()
edge_cols = [c for c in ['u','v','key','geometry','length','highway','maxspeed','speed_kph','travel_time'] if c in edges_gdf.columns]
edges = edges_gdf[edge_cols].copy().set_crs(4326)
for c in ['StartLon','StartLat','EndLon','EndLat']:
    tsb[c] = pd.to_numeric(tsb[c], errors='coerce')
tsb = tsb.dropna(subset=['StartLon','StartLat','EndLon','EndLat']).copy()
tsb['geometry'] = tsb.apply(lambda r: LineString([(r['StartLon'], r['StartLat']), (r['EndLon'], r['EndLat'])]), axis=1)
tsb_g = gpd.GeoDataFrame(tsb, geometry='geometry', crs='EPSG:4326')
edges_3414 = edges.to_crs(3414)
tsb_buf = tsb_g.to_crs(3414).copy(); tsb_buf['geometry'] = tsb_buf.geometry.buffer(25)
hit = gpd.sjoin(edges_3414, tsb_buf[['geometry','speed_kph_obs']], how='left', predicate='intersects')
edge_speed = hit.groupby(['u','v','key'], as_index=False)['speed_kph_obs'].mean()
edges_obs = edges_3414.merge(edge_speed, on=['u','v','key'], how='left')
edges_obs['speed_ratio'] = edges_obs['speed_kph_obs'] / edges_obs['speed_kph']
print('Edges with observed speed:', edges_obs['speed_kph_obs'].notna().sum(),'/', len(edges_obs))

Edges with observed speed: 45004 / 45951


In [27]:
import numpy as np, statsmodels.api as sm
df = edges_obs.to_crs(4326).copy()
df['road_cat'] = df.get('highway', 'other')
df_fit = df.dropna(subset=['speed_kph_obs','speed_kph']).copy()
df_fit = df_fit[df_fit['speed_kph']>0]
df_fit['rho_proxy'] = 1 - (df_fit['speed_kph_obs']/df_fit['speed_kph']).clip(lower=0, upper=1)
fd_params = {}
for cat, sub in df_fit.groupby('road_cat'):
    if len(sub) < 50: continue
    y = (sub['speed_kph_obs']/sub['speed_kph']).clip(0,1)
    X = sm.add_constant(1 - sub['rho_proxy'])
    try:
        res = sm.OLS(y, X).fit()
        slope = res.params[1]
        fd_params[cat] = {'vf_med': float(sub['speed_kph'].median()),
                          'kj_scale': float(max(0.2, min(5.0, 1/slope if slope>0 else 1.0)))}
    except Exception:
        continue
fd_params if fd_params else {'note':'insufficient data; using defaults'}

TypeError: unhashable type: 'list'

In [28]:
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets
import plotly.express as px
import folium

def simulate_speeds(df_base, fd_params, demand_factor=1.0, rain_mm=0.0, alpha=0.015, beta=0.01):
    d = df_base.copy()
    d['vf'] = d['speed_kph']; d['kj'] = 1.0
    for cat, p in fd_params.items():
        d.loc[d['road_cat']==cat, 'vf'] = p.get('vf_med', d['speed_kph'])
        d.loc[d['road_cat']==cat, 'kj'] = p.get('kj_scale', 1.0)
    d['rho0'] = (1 - (d['speed_kph_obs']/d['vf']).clip(0,1)).fillna(0.2)
    vf_p = d['vf'] * (1 - alpha*rain_mm)
    kj_p = d['kj'] * (1 - beta*rain_mm)
    ratio = (1 - (demand_factor * d['rho0']) / kj_p).clip(lower=0)
    d['speed_sim'] = (vf_p * ratio).fillna(d['speed_kph'])
    d['ratio_sim'] = (d['speed_sim']/d['speed_kph']).clip(0, 2)
    return d

def make_map(dplot):
    m = folium.Map(location=(1.3521, 103.8198), zoom_start=12, control_scale=True)
    def colorize(r):
        import numpy as np
        if r is None or np.isnan(r): return '#999999'
        r = max(0.0, min(1.5, r))
        return '#d73027' if r<0.6 else ('#fc8d59' if r<0.85 else ('#fee08b' if r<1.05 else ('#91cf60' if r<1.2 else '#1a9850')))
    g = dplot[['geometry','ratio_sim']].to_crs(4326)
    folium.GeoJson(
        data=g.__geo_interface__,
        style_function=lambda feat: {'color': colorize(feat['properties']['ratio_sim']), 'weight': 2, 'opacity': 0.9}
    ).add_to(m)
    return m

base = df_fit.copy()
def dashboard(demand=1.0, rain=0.0, alpha=0.015, beta=0.01):
    sim = simulate_speeds(base, fd_params, demand_factor=demand, rain_mm=rain, alpha=alpha, beta=beta)
    display(make_map(sim))
    fig1 = px.histogram(sim, x='speed_sim', nbins=50, title='Simulated Speed (km/h)'); fig1.show()
    fig2 = px.scatter(sim.sample(min(5000, len(sim)), random_state=1), x='speed_kph', y='speed_sim', opacity=0.4,
                      title='Simulated vs Free-flow (sample)'); fig2.update_traces(marker=dict(size=5)); fig2.show()
    vmean = float(sim['speed_sim'].mean()); cshare = float((sim['ratio_sim']<0.8).mean())
    print(f"Mean simulated speed: {vmean:.1f} km/h | Share < 0.8×free-flow: {100*cshare:.1f}%")

widgets.interact(
    dashboard,
    demand=widgets.FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description='Demand ×'),
    rain=widgets.FloatSlider(value=0.0, min=0.0, max=30.0, step=0.5, description='Rain (mm)'),
    alpha=widgets.FloatSlider(value=0.015, min=0.0, max=0.05, step=0.001, description='α (vf sens)'),
    beta=widgets.FloatSlider(value=0.010, min=0.0, max=0.05, step=0.001, description='β (kj sens)')
)

interactive(children=(FloatSlider(value=1.0, description='Demand ×', max=1.5, min=0.5, step=0.05), FloatSlider…

<function __main__.dashboard(demand=1.0, rain=0.0, alpha=0.015, beta=0.01)>

In [7]:
edges_out = edges_obs.to_crs(4326)[['u','v','key','speed_kph','speed_kph_obs','speed_ratio','highway','length']]
edges_out.to_csv('edges_observed_speeds.csv', index=False)
print('Wrote edges_observed_speeds.csv')

Wrote edges_observed_speeds.csv


## New Implementation

In [14]:
# ===========================
# SINGAPORE TRAFFIC DASHBOARD
# ===========================
# Data: OSMnx (roads), LTA DataMall (TrafficSpeedBands v4), NEA rainfall (optional)
# Map: Folium layers (base, actual, simulated), intersections; Plotly histograms/scatter

import os, math, json, datetime as dt
import numpy as np
import pandas as pd
import requests

import osmnx as ox
import networkx as nx
import geopandas as gpd
from shapely.geometry import LineString, Point

import folium
from branca.colormap import LinearColormap
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, HTML

# --------------------------
# 0) CONFIG / KEYS
# --------------------------
LTA_ACCOUNT_KEY = os.getenv("LTA_ACCOUNT_KEY", "").strip()  # <-- set env var or paste directly
NEA_API_KEY     = os.getenv("DATA_GOV_SG_API_KEY", "").strip()

if not LTA_ACCOUNT_KEY:
    raise RuntimeError("Missing LTA_ACCOUNT_KEY. Set env var or paste your key into LTA_ACCOUNT_KEY above.")

# --------------------------
# 1) OSMnx road network
# --------------------------
ox.settings.use_cache = True
G = ox.graph_from_place("Singapore", network_type="drive")
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
print(f"OSMnx graph → nodes: {G.number_of_nodes():,}, edges: {G.number_of_edges():,}")

# Edges GeoDataFrame (WGS84)
edges_gdf = ox.graph_to_gdfs(G, nodes=False, fill_edge_geometry=True)
if not {"u","v","key"}.issubset(edges_gdf.columns):
    edges_gdf = edges_gdf.reset_index()
edge_cols = [c for c in ["u","v","key","geometry","length","highway","maxspeed","speed_kph","travel_time"] if c in edges_gdf.columns]
edges_all = edges_gdf[edge_cols].copy()
edges_all = edges_all.set_crs(4326) if edges_all.crs is None else edges_all.to_crs(4326)

# Normalize road class to a hashable string
def _norm_cat(x):
    if isinstance(x, (list, tuple, set)):
        for e in x:
            if isinstance(e, str) and e:
                return e
        return "other"
    return "other" if x is None else str(x)
edges_all["road_cat"] = edges_all["highway"].apply(_norm_cat) if "highway" in edges_all.columns else "other"

# Nodes (for intersections layer)
nodes_gdf = ox.graph_to_gdfs(G, nodes=True, edges=False)
nodes_gdf = nodes_gdf.set_crs(4326) if nodes_gdf.crs is None else nodes_gdf.to_crs(4326)
deg = dict(G.degree())
nodes_gdf["degree"] = nodes_gdf.index.map(deg.get)
nodes_intersections = nodes_gdf[nodes_gdf["degree"].fillna(0) >= 3].copy()

# --------------------------
# 2) LTA: Traffic Speed Bands v4 (actual observations)
# --------------------------
headers = {"AccountKey": LTA_ACCOUNT_KEY}
URL_TSB = "https://datamall2.mytransport.sg/ltaodataservice/v4/TrafficSpeedBands"
URL_INC = "https://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents"

def fetch_all(url, headers, max_pages=50):
    out, next_url, n = [], url, 0
    while next_url and n < max_pages:
        r = requests.get(next_url, headers=headers, timeout=30)
        r.raise_for_status()
        j = r.json()
        out.extend(j.get("value", []))
        next_url = j.get("@odata.nextLink") or j.get("odata.nextLink")
        n += 1
    return pd.json_normalize(out)

tsb = fetch_all(URL_TSB, headers=headers)
inc = fetch_all(URL_INC, headers=headers)
print(f"LTA → TSB rows: {len(tsb):,} | Incidents: {len(inc):,}")

# Convert bands to mid-speed if min/max aren’t present
if {"MinimumSpeed","MaximumSpeed"}.issubset(tsb.columns):
    tsb["speed_kph_obs"] = (tsb["MinimumSpeed"].astype(float) + tsb["MaximumSpeed"].astype(float)) / 2.0
else:
    band_mid = {1:5,2:15,3:25,4:35,5:45,6:55,7:65,8:75}
    tsb["speed_kph_obs"] = tsb["SpeedBand"].map(band_mid).astype(float)

# --------------------------
# 3) NEA rainfall (optional, used as scenario input)
# --------------------------
def get_latest_rain(NEA_API_KEY=None):
    params = {"date_time": dt.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S")}
    headers_nea = {"api-key": NEA_API_KEY} if NEA_API_KEY else {}
    url = "https://api.data.gov.sg/v1/environment/rainfall"
    try:
        j = requests.get(url, params=params, headers=headers_nea, timeout=30).json()
        readings = pd.json_normalize(j["items"][0]["readings"])
        stations = pd.json_normalize(j["metadata"]["stations"])
        rain = readings.rename(columns={"value":"rain_mm"})[["station_id","rain_mm"]].merge(
            stations[["id","name","device_id","location.latitude","location.longitude"]],
            left_on="station_id", right_on="id", how="left"
        )
        return rain
    except Exception as e:
        print("NEA rainfall fetch failed (continuing without it):", e)
        return pd.DataFrame()

rain_df = get_latest_rain(NEA_API_KEY)
if not rain_df.empty:
    print(f"NEA rainfall stations: {len(rain_df)} (mm at ~5-min cadence)")

# --------------------------
# 4) Map-matching: TSB → edges (buffered intersects; LEFT merge to keep all edges)
# --------------------------
# Build TSB segment lines
for c in ["StartLon","StartLat","EndLon","EndLat"]:
    tsb[c] = pd.to_numeric(tsb[c], errors="coerce")
tsb_clean = tsb.dropna(subset=["StartLon","StartLat","EndLon","EndLat"]).copy()
tsb_clean["geometry"] = tsb_clean.apply(
    lambda r: LineString([(float(r["StartLon"]), float(r["StartLat"])),
                          (float(r["EndLon"]),   float(r["EndLat"]))]),
    axis=1
)
tsb_g = gpd.GeoDataFrame(tsb_clean, geometry="geometry", crs="EPSG:4326")

# Project to metric CRS (SVY21 / EPSG:3414)
edges_3414 = edges_all.to_crs(3414)
tsb_3414   = tsb_g.to_crs(3414)

# Buffer TSB lines and spatial join
tsb_buf = tsb_3414.copy()
tsb_buf["geometry"] = tsb_buf.geometry.buffer(25)  # meters (tune 20–40m if needed)

# GeoPandas spatial join (requires rtree or pygeos):
hit = gpd.sjoin(edges_3414, tsb_buf[["geometry","speed_kph_obs"]], how="left", predicate="intersects")
edge_speed = hit.groupby(["u","v","key"], as_index=False)["speed_kph_obs"].mean()

# Merge back to ALL edges (LEFT) to keep full network
edges_obs = edges_3414.merge(edge_speed, on=["u","v","key"], how="left")
edges_obs["observed"]    = edges_obs["speed_kph_obs"].notna()
edges_obs["speed_ratio"] = (edges_obs["speed_kph_obs"] / edges_obs["speed_kph"]).replace([np.inf,-np.inf], np.nan)

# For mapping
edges_map = edges_obs.to_crs(4326)
nodes_map = nodes_intersections.to_crs(4326)

print("Edges total:", len(edges_map),
      "| With observed speeds:", int(edges_map["observed"].sum()))

# --------------------------
# 5) Simple per-class FD params (for simulation)
# --------------------------
# Fit a very small Greenshields-style proxy per road class from the observed subset
df_fit = edges_map.dropna(subset=["speed_kph_obs","speed_kph"]).copy()
df_fit = df_fit[df_fit["speed_kph"] > 0]
df_fit["rho_proxy"] = 1 - (df_fit["speed_kph_obs"]/df_fit["speed_kph"]).clip(0,1)

fd_params = {}
for cat, sub in df_fit.groupby("road_cat"):
    if len(sub) < 50:
        continue
    # y ≈ const + slope*(1 - rho_proxy), slope ~ 1/kj (very rough)
    y = (sub["speed_kph_obs"]/sub["speed_kph"]).clip(0,1)
    X = np.vstack([np.ones(len(sub)), (1 - sub["rho_proxy"]).values]).T
    # OLS
    beta_hat, _, _, _ = np.linalg.lstsq(X, y.values, rcond=None)
    slope = float(beta_hat[1])
    vf_med  = float(np.median(sub["speed_kph"]))
    kj_scale = float(max(0.2, min(5.0, 1.0/slope if slope > 1e-6 else 1.0)))
    fd_params[cat] = {"vf_med": vf_med, "kj_scale": kj_scale}

if not fd_params:
    # fallback default if not enough data
    fd_params = {"other": {"vf_med": float(np.nanmedian(edges_map["speed_kph"])), "kj_scale": 1.0}}

# --------------------------
# 6) Scenario engine (demand, rain) and map layers
# --------------------------
def simulate_speeds(df_base, fd_params, demand_factor=1.0, rain_mm=0.0, alpha=0.015, beta=0.01):
    d = df_base.copy()
    d["vf"] = d["speed_kph"].fillna(30.0)
    d["kj"] = 1.0
    # apply per-class params
    if isinstance(fd_params, dict):
        for cat, p in fd_params.items():
            mask = (d["road_cat"] == cat)
            if "vf_med" in p: d.loc[mask, "vf"] = p["vf_med"]
            if "kj_scale" in p: d.loc[mask, "kj"] = p["kj_scale"]
    # baseline proxy from observation if available
    chi_obs = (d["speed_kph_obs"] / d["vf"]).clip(0,1)
    d["rho0"] = (1 - chi_obs).fillna(0.2)
    # rain perturbation
    vf_p = d["vf"] * (1 - alpha*rain_mm)
    kj_p = d["kj"] * (1 - beta*rain_mm)
    ratio = (1 - (demand_factor * d["rho0"]) / kj_p).clip(lower=0)
    d["speed_sim"] = (vf_p * ratio).fillna(d["speed_kph"])
    d["ratio_sim"] = (d["speed_sim"] / d["speed_kph"]).replace([np.inf, -np.inf], np.nan).clip(0, 2)
    return d

base_all = edges_map.copy()

ratio_cmap = LinearColormap(
    ["#d73027","#fc8d59","#fee08b","#91cf60","#1a9850"], vmin=0.5, vmax=1.2
).to_step(index=[0.5,0.6,0.85,1.05,1.2])

def _edge_geojson(gdf, value_col, name, line_weight=2, show=True):
    layer = folium.FeatureGroup(name=name, show=show)
    def _style(feat):
        val = feat["properties"].get(value_col)
        if val is None or (isinstance(val,float) and math.isnan(val)):
            return {"color":"#999999","weight":1,"opacity":0.6}
        return {"color": ratio_cmap(val), "weight": line_weight, "opacity": 0.9}

    fields = ["road_cat","speed_kph","speed_kph_obs",value_col,"u","v","key"]
    fields = [f for f in fields if f in gdf.columns]
    folium.GeoJson(
        data=gdf[[*(fields), "geometry"]].__geo_interface__,
        style_function=_style,
        tooltip=folium.GeoJsonTooltip(fields=fields, aliases=fields, localize=True),
        name=name
    ).add_to(layer)
    return layer

def _nodes_layer(nodes, name="Intersections (deg≥3)", show=False):
    layer = folium.FeatureGroup(name=name, show=show)
    for idx, r in nodes.iterrows():
        geom = r.geometry
        if geom and isinstance(geom, Point):
            folium.CircleMarker(
                location=(geom.y, geom.x),
                radius=2.5, color="#0044cc", fill=True, fill_opacity=0.7,
                tooltip=f"node: {idx} | degree: {int(r.get('degree',0))}"
            ).add_to(layer)
    return layer

def build_map(demand=1.0, rain=0.0, alpha=0.015, beta=0.01):
    # Actual layer (observed where available)
    edges_actual = base_all.copy()
    edges_actual["ratio_actual"] = (edges_actual["speed_kph_obs"] / edges_actual["speed_kph"]).replace([np.inf,-np.inf], np.nan)

    # Simulated layer
    sim = simulate_speeds(base_all, fd_params, demand_factor=demand, rain_mm=rain, alpha=alpha, beta=beta)

    m = folium.Map(location=(1.3521, 103.8198), zoom_start=12, control_scale=True, tiles="cartodbpositron")

    # Base gray network
    gray = base_all.copy()
    gray["ratio_gray"] = np.nan
    _edge_geojson(gray, "ratio_gray", name="Base network (gray)", line_weight=1, show=True).add_to(m)

    # Actual observed conditions
    _edge_geojson(edges_actual, "ratio_actual", name="Actual (LTA TSB)", line_weight=3, show=True).add_to(m)

    # Simulated conditions (toggle)
    _edge_geojson(sim, "ratio_sim", name="Simulated (FD demand/rain)", line_weight=2, show=False).add_to(m)

    # Intersections layer
    _nodes_layer(nodes_map, show=False).add_to(m)

    ratio_cmap.caption = "Speed ratio (colour = value / free-flow)"
    ratio_cmap.add_to(m)
    folium.LayerControl(collapsed=False).add_to(m)
    return m

# --------------------------
# 7) Dashboard: map + charts
# --------------------------
def dashboard(demand=1.0, rain=0.0, alpha=0.015, beta=0.01):
    m = build_map(demand, rain, alpha, beta)
    display(m)

    sim = simulate_speeds(base_all, fd_params, demand_factor=demand, rain_mm=rain, alpha=alpha, beta=beta)
    # Charts
    fig1 = px.histogram(sim, x="speed_sim", nbins=60, title="Simulated Speed (km/h)")
    fig1.show()

    sample = sim.sample(min(8000, len(sim)), random_state=1)
    fig2 = px.scatter(sample, x="speed_kph", y="speed_sim", opacity=0.35,
                      title="Simulated vs Free-flow (sample)",
                      labels={"speed_kph":"Free-flow (km/h)", "speed_sim":"Simulated (km/h)"})
    fig2.update_traces(marker=dict(size=4))
    fig2.show()

    vmean = float(sim["speed_sim"].mean())
    cshare = float((sim["ratio_sim"] < 0.8).mean())
    print(f"Mean simulated speed: {vmean:.1f} km/h | Share < 0.8×free-flow: {100*cshare:.1f}%")

# Show initial map
_ = dashboard(1.0, 0.0, 0.015, 0.01)

# Interactive controls (toggle simulated layer via LayerControl on the map)
widgets.interact(
    lambda demand, rain, alpha, beta: dashboard(demand, rain, alpha, beta),
    demand=widgets.FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description="Demand ×"),
    rain=widgets.FloatSlider(value=0.0, min=0.0, max=30.0, step=0.5, description="Rain (mm)"),
    alpha=widgets.FloatSlider(value=0.015, min=0.0, max=0.05, step=0.001, description="α (vf sens)"),
    beta=widgets.FloatSlider(value=0.010, min=0.0, max=0.05, step=0.001, description="β (kj sens)")
);


RuntimeError: Missing LTA_ACCOUNT_KEY. Set env var or paste your key into LTA_ACCOUNT_KEY above.